In [1]:
pip install tflearn

     |████████████████████████████████| 107 kB 5.5 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=682dcfdac25c39434be815311633a09f15c98d260ea386a05b169546f677cb57
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn


In [2]:
#@title Default title text
# Libraries needed for NLP
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# Libraries needed for Tensorflow processing
import tensorflow as tf
import numpy as np
from tensorflow.python.framework import ops
import random
import json
import tflearn

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
from google.colab import files
files.upload()

Saving intents.json to intents.json


{'intents.json': b'\n{"intents": [\n        {"tag": "vc name",\n        "patterns": ["who is president of csjmu","who is vc of csjmu", "What is the name of vc of csjmu", "who is vice chancellor of csjmu", "who is vice chancellor of chhatrapati sahu ji maharaj university "],\n        "responses": ["Prof. Vinay Kumar Pathak is the president of and vice chancellor of csjm university","Prof. Vinay Kumar Pathak", "Prof. Vinay Kumar Pathak is the vice chancellor of csjm university", "Prof. Vinay Kumar Pathak is the vice chancellor of chhatrapati sahu ji maharaj university"],\n        "context_filter": "vc"\n        } ,\n        {"tag": "registrar",\n            "patterns": ["WHO IS REGISTRAR OF CSJMU","what is the name of registrar of csjmu", "registrar of csjmu", "registrar of chhatrapati sahu ji maharaj university"],\n           "responses": ["Dr. Anil Kumar Yadav","Dr. Anil Kumar Yadav is the registrar of csjmu","Dr. Anil Kumar Yadav is the registrar of chhatrapati sahu ji maharaj univers

In [4]:
# import our chat-bot intents file
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [5]:
intents

{'intents': [{'context_filter': 'vc',
   'patterns': ['who is president of csjmu',
    'who is vc of csjmu',
    'What is the name of vc of csjmu',
    'who is vice chancellor of csjmu',
    'who is vice chancellor of chhatrapati sahu ji maharaj university '],
   'responses': ['Prof. Vinay Kumar Pathak is the president of and vice chancellor of csjm university',
    'Prof. Vinay Kumar Pathak',
    'Prof. Vinay Kumar Pathak is the vice chancellor of csjm university',
    'Prof. Vinay Kumar Pathak is the vice chancellor of chhatrapati sahu ji maharaj university'],
   'tag': 'vc name'},
  {'context_filter': 'registrar',
   'patterns': ['WHO IS REGISTRAR OF CSJMU',
    'what is the name of registrar of csjmu',
    'registrar of csjmu',
    'registrar of chhatrapati sahu ji maharaj university'],
   'responses': ['Dr. Anil Kumar Yadav',
    'Dr. Anil Kumar Yadav is the registrar of csjmu',
    'Dr. Anil Kumar Yadav is the registrar of chhatrapati sahu ji maharaj university'],
   'tag': 'regi

In [6]:
words = []
classes = []
documents = []
ignore = ['?']
# loop through each sentence in the intent's patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each and every word in the sentence
        w = nltk.word_tokenize(pattern)
        # add word to the words list
        words.extend(w)
        # add word(s) to documents
        documents.append((w, intent['tag']))
        # add tags to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [7]:
# Perform stemming and lower each word as well as remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

# remove duplicate classes
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

1215 documents
129 classes [' ', ' contact', ' vocational', 'admission', 'advance lab', 'agriculture ', 'antiragging', 'apply for addmission', 'arts', 'auditorium', 'award', 'ayurvedic', 'bank', 'bca', 'branches', 'btech', 'business', 'ca', 'cafeteria', 'chemistry', 'circulars', 'college', 'commerce', 'complex', 'computer centre', 'contact', 'contact director', 'contacts', 'copyright', 'core lab', 'council', 'counselling', 'course', 'course cse', 'courses', 'covid', 'cse course', 'cse seat', 'cse staff', 'cse student', 'csjmu', 'csjmu ', 'csjmuu', 'curriculum', 'degree', 'departments', 'director', 'director appoint', 'director name', 'director qualification', 'disclaimer', 'education', 'engineering', 'entrance exam', 'events', 'faculty', 'faulty', 'fees', 'finance officer', 'fo', 'form', 'gallery', 'governer', 'grievance', 'guest house', 'hall', 'helipad', 'hod bio informatics', 'hod ca', 'hod che', 'hod chemistry', 'hod cse', 'hod ece', 'hod maths', 'hod mee', 'hospital', 'hostel', 'h

In [8]:
# create training data
training = []
output = []
# create an empty array for output
output_empty = [0] * len(classes)

# create training set, bag of words for each sentence
for doc in documents:
    # initialize bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stemming each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is '1' for current tag and '0' for rest of other tags
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffling features and turning it into np.array
random.shuffle(training)
training = np.array(training)

# creating training lists
train_x = list(training[:,0])
train_y = list(training[:,1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [11]:
# resetting underlying graph data
ops.reset_default_graph()

# Building neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Defining model and setting up tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

# Start training
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 151999  | total loss: 0.00114 | time: 0.980s
| Adam | epoch: 1000 | loss: 0.00114 - acc: 0.9989 -- iter: 1208/1215
Training Step: 152000  | total loss: 0.00103 | time: 0.986s
| Adam | epoch: 1000 | loss: 0.00103 - acc: 0.9990 -- iter: 1215/1215
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [12]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [13]:
# restoring all the data structures
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [14]:
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [15]:
# load the saved model
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from /content/model.tflearn


In [16]:
def clean_up_sentence(sentence):
    # tokenizing the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stemming each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# returning bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenizing the pattern
    sentence_words = clean_up_sentence(sentence)
    # generating bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [17]:
ERROR_THRESHOLD = 0.30
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    return print(random.choice(i['responses']))

            results.pop(0)

In [18]:
classify('vc of csjmu')

[('librarian', 0.67389417), ('vc name', 0.3260082)]

In [34]:
response('name computer science and engineering hod')

Dr. Deepak Kumar Verma


In [20]:
response('how many branches are there in btech')

CSE,CSE(AI),ECE,CHEMICAL


In [22]:
response('name of hod of ece')

Er. Niraj kumar


In [36]:
response('name ece hod')

Er. Niraj kumar is the head of department of electronics and communication engineering department


In [37]:
response('who is director of engineering')

Dr. Brishti Mitra


In [38]:
response('vice chancellor of csjmu')

Prof. Vinay Kumar Pathak


In [29]:
#Some of other context free responses.
response('courses in csjmu')

Instagram - https://www.instagram.com/csjmuknp/


In [30]:
response('registrar of csjmu')

Dr. Anil Kumar Yadav is the registrar of csjmu


In [31]:
response('seats in pg diploma')

there are 60 seats


In [39]:
#Adding some context to the conversation i.e. Contexualization for altering question and intents etc.
# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                          # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return print(random.choice(i['responses']))

            results.pop(0)

In [40]:
response('who is vc of csjmu')